<a href="https://colab.research.google.com/github/dark-syndrome/ANN-for-prediction-of-Moisture-Ratio/blob/master/majorProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function

import pathlib
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

data = pd.read_csv('majorProject.csv')
print(tf.__version__)

FileNotFoundError: ignored

In [ ]:
data

In [ ]:
#Cleaning data
data.isna().sum()

In [ ]:
data.dropna()

In [ ]:
train_data = data.sample(frac = 0.8, random_state=0)
test_data = data.drop(train_data.index)

In [ ]:
sns.pairplot(train_data[["TIME", "POWER", "MOISTURE CONTENT (INITIAL)", "MOISTURE RATIO"]], diag_kind="kde")

In [ ]:
train_stats = train_data.describe()
train_stats.pop("MOISTURE RATIO")
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_data.pop("MOISTURE RATIO")
test_labels = test_data.pop("MOISTURE RATIO")

In [ ]:
#Now we are going to normalize the data remaining.
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_data)
normed_test_data = norm(test_data)

#similarly when we pass new values to this model we need to
 #  normalize all the input data we are giving"""

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_data.keys())]),
    layers.Dense(64, activation= tf.nn.relu),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.SGD()

  model.compile(loss='mse',
                optimizer = optimizer,
                metrics = ['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
print(example_result)

In [ ]:
#Training our model
#Display training process by printig one * for each cycle of training
class PrintStar(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('*', end='')
EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs = EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintStar()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error[MOISTURE RATIO]')
  plt.plot(hist['epoch'], hist['mae'],
           label = 'Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'validation error')
  plt.legend()
  plt.ylim([0,1])

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean square Error [$MOISTURE RATIO ^ 2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label = 'Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'validation error')
  plt.legend()
  plt.ylim([0,0.1])

plot_history(history)

In [ ]:
#model = build_model()

#early_stop = keras.callbacks.EarlyStopping(monitor= 'val_loss', patience= 10)

#history = model.fit(normed_train_data, train_labels, epochs = EPOCHS,
                   # validation_split =0.2, verbose=0, callbacks=[early_stop, PrintStar()])

#plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
print("Testing set Mean Abs Error: {} MOISTURE RATIO".format(mae))

In [ ]:
#Finally I am demonstrating the predictions here

test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True values [MOISTURE RATIO]')
plt.ylabel('Predictions [MOISTURE RATIO]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins =25)
plt.xlabel("Prediction error of MOISTURE RATIO")
_ = plt.ylabel("count")